# Importing Libraries

In [30]:
#import os
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
#from sklearn import svm
#import sklearn.cross_validation
from sklearn.model_selection import train_test_split
from numpy import random
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
from sklearn import svm
import warnings
warnings.filterwarnings('ignore')
import gensim
from gensim.models import Word2Vec

# Loading the Train Data

In [31]:
data = pd.read_excel('Finish SL Train Data.xlsx',sheet_name="Train1")
data.columns

Index(['Mention Content', 'Level1', 'Level2'], dtype='object')

# Data Cleaning

In [32]:
print("\t###### Encoding Target Variable ######\t\n")    
lb_make = LabelEncoder()
data["lable_code"] = lb_make.fit_transform(data['Level2'].astype(str))

data["lable_code"].describe()

	###### Encoding Target Variable ######	



count    11709.000000
mean        76.212999
std         27.959768
min          0.000000
25%         58.000000
50%         80.000000
75%         99.000000
max        139.000000
Name: lable_code, dtype: float64

In [33]:
# converting all messages to lower case
data['Mention Content']=data['Mention Content'].str.lower()

In [144]:
 # Replace email addresses with 'email'
data['Mention Content']=data['Mention Content'].str.replace(r'^.+@[^\.].*\.[a-z]{2,}$','emailaddress')

In [145]:
# Replace URLs with 'webaddress'
# Replace money symbols with 'moneysymb'(£ can be typed with ALT + 156# Replace money symbols with 'moneysymb'(£ can be typed with ALT + 156
data['Mention Content']=data['Mention Content'].str.replace(r'^http\://a[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$',
                                            'webaddress')


In [146]:
 # Replace money symbols with 'moneysymb'(£ can be typed with ALT + 156
data['Mention Content']=data['Mention Content'].str.replace(r'£|\$','dollers')


In [147]:
# Replace 10 didgit phone numbers(formats include paranthesis, spaces, no spaces, dashes) with 'phonenumber'
data['Mention Content']=data['Mention Content'].str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{d}','phonenumber')

In [148]:
# replace numbers with 'numbr'
data['Mention Content']=data['Mention Content'].str.replace(r'\d+(\.\d+)?','numbr')

In [149]:
# remove punctuation
data['Mention Content']=data['Mention Content'].str.replace(r'[^\w\d\s]',' ')

In [150]:
# Replace whitespace between terms with a single space
data['Mention Content']=data['Mention Content'].str.replace(r'\s+',' ')

In [151]:
# Remove Leading and trailing whitespace
data['Mention Content']=data['Mention Content'].str.replace(r'^\s+|\s+?$',' ')

In [34]:
data_x = data['Mention Content'].values.tolist()
data_y = np.asarray(data['lable_code'],dtype=int)

In [35]:
stopset = set(stopwords.words('english'))


In [36]:
print("\t###### Converting Text to Feature Vector ######\t\n")    
cv =  CountVectorizer(lowercase=True,stop_words=stopset, ngram_range=(1,3))
X_traincv = cv.fit_transform(str(e) for e in data_x)

	###### Converting Text to Feature Vector ######	



# Train Test Split and Model Building

In [9]:
#train_x,test_x,train_y,test_y=train_test_split(X_traincv,data_y,test_size=0.4)

In [37]:
x_train, x_test, y_train, y_test=train_test_split(X_traincv, data_y ,random_state=47, test_size=0.4)

In [38]:
#from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
#from sklearn.naive_bayes import GaussianNB
#from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import ComplementNB

In [17]:
# Gradient Boosting Classifier
gbt=GradientBoostingClassifier()
gbt.fit(x_train,y_train)
pred_y=gbt.predict(x_test)
print('accuracy score',accuracy_score(y_test,pred_y))
print("confusion matrix")
print(confusion_matrix(y_test,pred_y))
print('classification',classification_report(y_test,pred_y)) 
# Probablity
print('gbt_prob',gbt.predict_proba(x_test))

KeyboardInterrupt: 

In [39]:
# SGDClassifier
sgd=SGDClassifier(loss='hinge',class_weight='balanced')
sgd.fit(x_train,y_train)
pred_y=sgd.predict(x_test)
print('accuracy score',accuracy_score(y_test,pred_y))
print("confusion matrix")
print(confusion_matrix(y_test,pred_y))
print('classification',classification_report(y_test,pred_y))
# Probablity
print('sgd_prob',sgd.predict_proba(x_test))


accuracy score 0.7585397096498719
confusion matrix
[[93  0  0 ...  0  0  0]
 [ 0  1  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  1  0  0]
 [ 0  0  0 ...  0  2  0]
 [ 0  0  0 ...  1  0  3]]
classification               precision    recall  f1-score   support

           0       0.89      0.82      0.86       113
           1       0.33      1.00      0.50         1
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         1
           4       0.57      0.22      0.32        18
           5       0.46      0.55      0.50        29
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.

AttributeError: probability estimates are not available for loss='hinge'

In [87]:
# SGD Classifier Cross Validation
#score=cross_val_score(sgd,x_train,y_train,cv=10)
#print('cv score',np.mean(score))

In [40]:
# ComplementNB
cnb=ComplementNB()
cnb.fit(x_train,y_train)
pred_y=cnb.predict(x_test)
print('accuracy score',accuracy_score(y_test,pred_y))
print("confusion matrix")
print(confusion_matrix(y_test,pred_y))
print('classification',classification_report(y_test,pred_y))
# Probablity
print('cnb_prob',cnb.predict_proba(x_test))


accuracy score 0.7506404782237404
confusion matrix
[[96  0  0 ...  0  0  0]
 [ 0  1  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  2  0]
 [ 0  0  0 ...  0  0  0]]
classification               precision    recall  f1-score   support

           0       0.71      0.85      0.77       113
           1       0.50      1.00      0.67         1
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         1
           4       0.33      0.06      0.10        18
           5       0.63      0.66      0.64        29
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         4
          11       0.00      0.00      0.00         0
          15       0.47      0.92      0.62        24
          16       0.00      0.00      0.00         1
          17       0.

In [12]:
# CNB Classifier Cross Validation
#score=cross_val_score(cnb,x_train,y_train,cv=10)
#print('cv score',np.mean(score))

In [13]:
# Extra Trees Classifier
etc=ExtraTreesClassifier()
etc.fit(x_train,y_train)
pred_y=etc.predict(x_test)
print('accuracy score',accuracy_score(y_test,pred_y))
print("confusion matrix")
print(confusion_matrix(y_test,pred_y))
print('classification',classification_report(y_test,pred_y))
# Probablity
print('etc_prob',etc.predict_proba(x_test))

KeyboardInterrupt: 

In [41]:
# MultinomialNB
mnb=MultinomialNB()
mnb.fit(x_train,y_train)
pred_y=mnb.predict(x_test)
print('accuracy score',accuracy_score(y_test,pred_y))
print("confusion matrix")
print(confusion_matrix(y_test,pred_y))
print('classification',classification_report(y_test,pred_y))
mnb.predict_proba(x_test)
# Probablity
print('mnb_prob',mnb.predict_proba(x_test))

accuracy score 0.7359094790777113
confusion matrix
[[94  0  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
classification               precision    recall  f1-score   support

           0       0.85      0.83      0.84       113
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00        18
           5       1.00      0.21      0.34        29
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         4
          15       0.79      0.92      0.85        24
          16       0.00      0.00      0.00         1
          21       0.36      0.56      0.44       130
          22       1.

In [12]:
#print("\t###### Building  Model ######\t\n")    
#nbayes = MultinomialNB()
#nbayes.fit(train_x,train_y)
#from sklearn.ensemble import RandomForestClassifier

#pred=nbayes.predict(test_x)
#accuracy_score(pred,test_y)

# Loading the Test Data

In [42]:
test_data = pd.read_excel('Finish SL Train Data.xlsx',sheet_name="Test1")

test_data1 = test_data['Review']

In [43]:
test_data.columns

Index(['ID', 'Review'], dtype='object')

# Prediction on Test Data(Best Model)

In [44]:
print("\t###### Converting Text to Feature Vector ######\t\n")
test_prediction = cv.transform(str(i) for i in test_data1)

	###### Converting Text to Feature Vector ######	



In [45]:
# Prediction on best Model
print("\t###### Predicting SMS Text ######\t\n")
test_prediction = sgd.predict(test_prediction)

	###### Predicting SMS Text ######	



In [46]:
predict_labels = pd.Series(lb_make.inverse_transform(test_prediction))

In [47]:
#output1=output1.drop(['Level1_x'], axis = 1)

In [48]:
output1 = pd.DataFrame({'ID':test_data['ID'], 'Text1':test_data['Review'],
                       'Level2':predict_labels})

#output.to_excel('Lysol_p.xlsx',index=False)

In [49]:
output1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 0 to 1066
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      1067 non-null   int64 
 1   Text1   1062 non-null   object
 2   Level2  1067 non-null   object
dtypes: int64(1), object(2)
memory usage: 25.1+ KB


# Leveling the Data

In [50]:
test1 = pd.read_excel('Finish SL Train Data.xlsx',sheet_name="t3")

In [51]:
test1.columns

Index(['Level1', 'Level2'], dtype='object')

In [52]:
output1=pd.merge(output1,test1,on='Level2',how='left')


In [53]:
output1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1067 entries, 0 to 1066
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      1067 non-null   int64 
 1   Text1   1062 non-null   object
 2   Level2  1067 non-null   object
 3   Level1  759 non-null    object
dtypes: int64(1), object(3)
memory usage: 41.7+ KB


In [54]:
pd.crosstab(index=output1["Level2"],columns="count") 

col_0                               count
Level2                                   
#SkipTheRinse                          16
Advertisement Negative Feedback         1
Advertisement Positive Feedback         1
Bad Experience                          1
Best Product                            1
Best Technology                         1
Biodegrable                             1
Boycott                                16
Broken/Crushed Product                  1
Chemical Product                        2
Claims of Counterfeit Products          1
Cleans Well                            10
Contest (#VitrinlereÖzgürlük)          15
Counterfit Product                      1
Coupon Request                          1
Crushed/Broken during shipment          2
Damage the dishwasher                   1
Damage to Dishes                        1
Did not clean                           3
Dispensing Issue                        7
Does Not Clean                          2
Does not Clean                          1
Expensive                               2
Expired                                 4
Expired Product                         1
Extra Packagng                          5
Eye irrition                           24
Finish Challenge(SkipTheRinse)         15
Fragrance/Smell Issues                  2
General Brand Mention                  16
General Comment                        66
General Product  Negative feedback      2
General Product Positive Feedabck       3
Gift not recived                       18
Junk                                  217
Leakage                                 1
Leaves Residue/Spots                    1
Leaves a Soap Taste                     1
Looking like product is dupilcate       1
Missing or incomplete contents          2
Negative Brand Perception               2
Offer Enquiry                           2
Order not recived                       1
Packaging Issue                         3
Political Issues                       52
Positive Brand Perception              46
Price                                   1
Product Deals                         472
Product Design Issues                   4
Refund                                  2
Sampler Box                             1
Service Complaint                       1
Short Count                             3
Suggesting Competitor                   1
Sustainability                          1
Water conservation                      1
Website Feedback                        1
What is the Price?                      4
Where to buy?                           2

In [55]:
output1[:50]

ID                                              Text1  \
0    1  ...embrace money off of popcorn, dumplings, on...   
1    2  Honestly the ember twin quantum also took me a...   
2    3  ... with my respect, all. Hepsiburada Greengap...   
3    4  ðŸ‘€ Set: Finish Power - Dishwasher Detergent ...   
4    5  The old Quantum Leap episodes. And when you fi...   
5    6  JenniferW wrote: â†‘Tue Feb 28, 2023 10:04 am ...   
6    7  quantAndHold wrote: â†‘Tue Feb 28, 2023 9:23 a...   
7    8  JenniferW wrote: â†‘Tue Feb 28, 2023 1:22 am S...   
8    9  Doom&Gloom wrote: â†‘ Fri Feb 24, 2023 2:13 pm...   
9   10          Is *The Quantum Archangel* worth reading?   
10  11  Ð ð ð ° ð ð ð ð ðµñ ð ððð ððð ñivil ñƒƒð>ðð¹ð¹...   
11  12  Doom&Gloom wrote: â†‘ Fri Feb 24, 2023 2:13 pm...   
12  13  Doom&Gloom wrote: â†‘ Fri Feb 24, 2023 2:13 pm...   
13  14  Iâ€™ve no idea what Dishwasher tablet manufact...   
14  15  Doom&Gloom wrote: â†‘ Fri Feb 24, 2023 2:13 pm...   
15  16  Switched from Finish pods ($71 per year) to Do...   
16  17  ... dishwasher. RTV Euro AGD Comment on the al...   
17  18  Just spent a few days turning an entire planet...   
18  19  @nikkid222 @nursekelsey Yes!! Especially if th...   
19  20  Not that at all. The other photo was taken at ...   
20  21  ...the toughest editions in the history of Run...   
21  22  ... Some of the waiting discounts; Sensodyne F...   
22  23  This. I get a better result from a $4 bottle o...   
23  24  RT @MaziUbaOfficial Finland based fraudster an...   
24  25  No because I can't quantum travel without cras...   
25  26                                  A quantum finish!   
26  27  Finish Powerball Quantum Ultimate Pellets for ...   
27  28  ... Finish Quantum Infinity Shine You can also...   
28  29  RT @bargainhdc This 100 pack of Finish Powerba...   
29  30  RT @bargainhdc This 100 pack of Finish Powerba...   
30  31  RT @bargainhdc This 100 pack of Finish Powerba...   
31  32  RT @bargainhdc This 100 pack of Finish Powerba...   
32  33  RT @bargainhdc This 100 pack of Finish Powerba...   
33  34  This 100 pack of Finish Powerball dishwasher t...   
34  35  ... you don't have to worry about spending mon...   
35  36  RT @Giftideas17 This 100 pack of Finish Powerb...   
36  37  RT @Giftideas17 This 100 pack of Finish Powerb...   
37  38  RT @Giftideas17 This 100 pack of Finish Powerb...   
38  39  RT @Giftideas17 This 100 pack of Finish Powerb...   
39  40  RT @Giftideas17 This 100 pack of Finish Powerb...   
40  41  RT @Giftideas17 This 100 pack of Finish Powerb...   
41  42  RT @Giftideas17 This 100 pack of Finish Powerb...   
42  43  This 100 pack of Finish Powerball dishwasher t...   
43  44  ... spending money on expensive, aggressive ch...   
44  45  ...used for centuries. Always produced without...   
45  46  ...used for centuries. Always produced without...   
46  47  ...used for centuries. Always produced without...   
47  48  ...used for centuries. Always produced without...   
48  49  ...used for centuries. Always produced without...   
49  50  ...used for centuries. Always produced without...   

                            Level2                  Level1  
0        Negative Brand Perception                     NaN  
1                     Eye irrition                     NaN  
2   Missing or incomplete contents       Product Condition  
3                    Product Deals    Actions & Promotions  
4                             Junk                     NaN  
5                    Product Deals    Actions & Promotions  
6                    Product Deals    Actions & Promotions  
7                    Product Deals    Actions & Promotions  
8                 Gift not recived     Consumer Experience  
9                          Boycott  Spam & Harmful Content  
10                            Junk                     NaN  
11                Gift not recived     Consumer Experience  
12                Gift not recived     Consumer Experience  
13                   Product Deals    

# Saving the  model

In [56]:
writer = pd.ExcelWriter(r'Finish SL feb 2 sgd.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
output1.to_excel(writer)
writer.close()


In [52]:
import googletrans
print(googletrans.LANGUAGES)

{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'latin', 'lv': 'latv